<a href="https://www.kaggle.com/code/tatianadzvinchuk/model-3-new?scriptVersionId=215821517" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Поточна версія моделі - це модель з найпростішою архітектурою.


In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Layer
from tensorflow.keras.models import Model

from transformers import TFBertForSequenceClassification, TFBertModel

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split


**Підготовка тренувальних даних**

In [3]:
data_path = '/kaggle/input/dataset-new/train_data.csv'
df = pd.read_csv(data_path)

# список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    df[column] = df[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(df['input_ids'].values)
attention_mask = np.stack(df['attention_masks'].values)
labels = np.array(df[LABEL_COLUMNS].values)
labels = labels.astype('float32')

In [4]:
# Розділення на тренувальну та тестову вибірки

train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

In [5]:
# Перевірка форматів даних 
print("input_ids dtype:", input_ids.dtype)
print("attention_mask dtype:", attention_mask.dtype)
print("labels dtype:", labels.dtype)

# Перевірка довжини даних
print("Lengths match:", len(input_ids) == len(attention_mask) == len(labels))

input_ids dtype: int64
attention_mask dtype: int64
labels dtype: float32
Lengths match: True


**Створення і навчання моделі**

In [6]:
# Кастомний шар для інтеграції з BERT
class BertLayer(Layer):
    def __init__(self, pretrained_model_name="bert-base-uncased", trainable=False, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        # Завантажуємо попередньо навчений BERT
        self.bert = TFBertModel.from_pretrained(pretrained_model_name)
        self.bert.trainable = trainable  # Заморожуємо або розморожуємо шари залежно від параметра trainable

    def call(self, inputs):
        # Вхідні дані: input_ids та attention_mask
        input_ids, attention_mask = inputs
        # Передаємо дані через BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Повертаємо тільки last_hidden_state

In [7]:
# Визначаємо вихідні дані для моделі
input_ids = Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(None,), dtype=tf.int32, name='attention_mask')

# Підключаємо кастомний шар BERT
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Додаємо класифікаційний шар
logits = Dense(len(LABEL_COLUMNS), activation='sigmoid', name='classifier')(bert_outputs[:, 0, :])  # Используем только [CLS]-токен

# Будуємо модель
model = Model(inputs=[input_ids, attention_mask], outputs=logits)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
# Компіляція моделі
model.compile(
    optimizer=Adam(learning_rate=5e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # BinaryCrossentropy для багатоміткової класифікації
    metrics=['accuracy']
)

In [9]:
# Визначення EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=3,          
    restore_best_weights=True  
)

# Навчання моделі
history = model.fit(
    {'input_ids': train_input_ids, 'attention_mask': train_attention_mask},
    train_labels,
    validation_data=(
        {'input_ids': val_input_ids, 'attention_mask': val_attention_mask},
        val_labels
    ),
    epochs=10,  # 
    batch_size=64,
    callbacks=[early_stopping]
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1298s 641ms/step - accuracy: 0.9927 - loss: 0.2342 - val_accuracy: 0.9941 - val_loss: 0.1413
Epoch 2/10
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1266s 635ms/step - accuracy: 0.9941 - loss: 0.1413 - val_accuracy: 0.9939 - val_loss: 0.1409
Epoch 3/10
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1265s 634ms/step - accuracy: 0.9942 - loss: 0.1384 - val_accuracy: 0.9941 - val_loss: 0.1406
Epoch 4/10
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1266s 635ms/step - accuracy: 0.9940 - loss: 0.1385 - val_accuracy: 0.9941 - val_loss: 0.1405
Epoch 5/10
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1263s 633ms/step - accuracy: 0.9939 - loss: 0.1387 - val_accuracy: 0.9941 - val_loss: 0.1404
Epoch 6/10
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1268s 636ms/step - accuracy: 0.9937 - loss: 0.1404 - val_accuracy: 0.9941 - val_loss: 0.1405
Epoch 7/10
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1266s 635ms/step - accuracy: 0.9943 - loss: 0.1400 - val_accuracy: 0.9941 - val_loss: 0.1403
Epoch 8/10
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1266s 634ms/step - accur

In [11]:
model.save("model_3.h5")


In [13]:
model.export("model_3new")


Saved artifact at 'model_3new'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, None), dtype=tf.int32, name='input_ids'), TensorSpec(shape=(None, None), dtype=tf.int32, name='attention_mask')]
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  139337193216432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139337193210272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139337193220656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139336967597472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139336967586736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139337193221360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139336967600464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139336967600992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139336967601520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139336967595888: TensorSp

In [14]:
!zip -r model_3new.zip model_3new


  adding: model_3new/ (stored 0%)
  adding: model_3new/assets/ (stored 0%)
  adding: model_3new/variables/ (stored 0%)
  adding: model_3new/variables/variables.index (deflated 76%)
  adding: model_3new/variables/variables.data-00000-of-00001

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


 (deflated 7%)
  adding: model_3new/saved_model.pb (deflated 93%)
  adding: model_3new/fingerprint.pb (stored 0%)


**Оцінка моделі на тренувальних даних (валідаційна вибірка)**

In [15]:
# Додання класу нетоксичних коментарів (всі нулі)
all_zeros_class = np.all(val_labels == 0, axis=1)
y_val_expanded = np.hstack((val_labels, all_zeros_class.reshape(-1, 1)))

# Прогнозування на валідаційній вибірці
predictions = model.predict({'input_ids': val_input_ids, 'attention_mask': val_attention_mask}, batch_size=64)
predicted_classes = (predictions > 0.5).astype(int)  # Бінарізація прогнозу

# Додання класу нетоксичних коментарів у прогноз
predicted_all_zeros_class = np.all(predicted_classes == 0, axis=1)
predicted_classes_expanded = np.hstack((predicted_classes, predicted_all_zeros_class.reshape(-1, 1)))

# Оцінка моделі
print("\nКласифікаційний звіт:\n")
print(classification_report(y_val_expanded, predicted_classes_expanded, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "non_toxic"
]))

# Побудова багатоміткової матриці помилок
conf_matrices = multilabel_confusion_matrix(y_val_expanded, predicted_classes_expanded)

# Приклад виводу (наприклад для "toxic")
print("Confusion matrix for 'toxic':")
print(conf_matrices[0])

499/499 ━━━━━━━━━━━━━━━━━━━━ 266s 524ms/step

Класифікаційний звіт:

               precision    recall  f1-score   support

        toxic       0.00      0.00      0.00      3056
 severe_toxic       0.00      0.00      0.00       321
      obscene       0.00      0.00      0.00      1715
       threat       0.00      0.00      0.00        74
       insult       0.00      0.00      0.00      1614
identity_hate       0.00      0.00      0.00       294
    non_toxic       0.90      1.00      0.95     28671

    micro avg       0.90      0.80      0.85     35745
    macro avg       0.13      0.14      0.14     35745
 weighted avg       0.72      0.80      0.76     35745
  samples avg       0.90      0.90      0.90     35745

Confusion matrix for 'toxic':
[[28859     0]
 [ 3056     0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Оцінка моделі на тестових даних**

In [16]:
data_path_test = '/kaggle/input/dataset-new/test_data.csv'
df_test = pd.read_csv(data_path_test)

# список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    df_test[column] = df_test[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
test_input_ids = np.stack(df_test['input_ids'].values)
test_attention_mask = np.stack(df_test['attention_masks'].values)
test_labels = np.array(df_test[LABEL_COLUMNS].values)
test_labels = test_labels.astype('float32')

In [17]:
# Додання класу нетоксичних коментарів (всі нулі)
all_zeros_class = np.all(test_labels == 0, axis=1)
y_test_expanded = np.hstack((test_labels, all_zeros_class.reshape(-1, 1)))

# Прогнозування на валідаційній вибірці
predictions_test = model.predict({'input_ids': test_input_ids, 'attention_mask': test_attention_mask}, batch_size=64)
predicted_classes_test = (predictions_test  > 0.5).astype(int)  # Бінарізація прогнозу

# Додання класу нетоксичних коментарів у прогноз
predicted_all_zeros_class = np.all(predicted_classes_test == 0, axis=1)
predicted_classes_expanded_test = np.hstack((predicted_classes_test, predicted_all_zeros_class.reshape(-1, 1)))

# Оцінка моделі
print("\nКласифікаційний звіт:\n")
print(classification_report(y_test_expanded, predicted_classes_expanded_test, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "non_toxic"
]))

# Побудова багатоміткової матриці помилок
conf_matrices = multilabel_confusion_matrix(y_test_expanded, predicted_classes_expanded_test)

# Приклад виводу (наприклад для "toxic")
print("Confusion matrix for 'toxic':")
print(conf_matrices[0])

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 513s 514ms/step

Класифікаційний звіт:

               precision    recall  f1-score   support

        toxic       0.00      0.00      0.00      6090
 severe_toxic       0.00      0.00      0.00       367
      obscene       0.00      0.00      0.00      3691
       threat       0.00      0.00      0.00       211
       insult       0.00      0.00      0.00      3427
identity_hate       0.00      0.00      0.00       712
    non_toxic       0.90      1.00      0.95     57735

    micro avg       0.90      0.80      0.85     72233
    macro avg       0.13      0.14      0.14     72233
 weighted avg       0.72      0.80      0.76     72233
  samples avg       0.90      0.90      0.90     72233

Confusion matrix for 'toxic':
[[57888     0]
 [ 6090     0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
